In [2]:
import time
import numpy as np
from qboard.cimsim.cimsim import CIMSim
from qboard.cimsim import tuner
import qboard
import qboard.cache
from qboard.translators import markowitz
from qboard import qubo

import os
os.chdir('/home/koritskiy/rqc/markowitz_partitioning')
os.getcwd()
from modules.partitioning import Partitioning

5

In [2]:
np.random.seed(4)
# Make console print look better
np.set_printoptions(precision=3,  # Digits after point
                    linewidth=170,  # Length of the line
                    suppress=True)  # Always fixed point notation (not scientific)


In [3]:
block_dim = 344


# define the file path
def path(name):
    return f"/home/koritskiy/rqc/markowitz_partitioning/data/test_matrices/{block_dim}/{name}{block_dim}.csv"


# download matrices
block_mat = np.genfromtxt(path("block_mat"), delimiter=",")
mix_permutation_mat = np.genfromtxt(path("mix_permutation_mat"), delimiter=",")
mixed_mat = np.genfromtxt(path("mixed_mat"), delimiter=",")

# Create object of partitioning task
part = Partitioning(mixed_mat, theta=50)
bqm = part.bqm

In [4]:
Q = part.qubo_mat
h, J = qubo.toising(Q)

In [7]:
import multiprocessing

params = qboard.cache.Parameters()
print(params[h, J])
def optimizer(h, J):
    params[h, J] = tuner.optimize(h, J)

p = multiprocessing.Process(target=optimizer, name="Optimizer", args=(h, J))
timeout = 100
p.start()
p.join(timeout)

# If still optimizing
if p.is_alive():
    p.terminate()
    p.join()
    

# solutions = qboard.cache.Solutions()
# if (h, J) in solutions:
#     spins_ising = [1 if _ else -1 for _ in solutions[h, J]]
#     energy_ising = qubo.ienergy(h, J, spins_ising)
# else:
# if not params[h, J]:

{'dt': 1.135932530023061e-05, 'sigma': 0.007007702672338741, 'alpha': 0.0, 'S': 11.896968981086516, 'D': -12.007289141982621, 'O': 0.0019393790409842787}




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Initial perc: (-41027.833837220045)	Final perc:(-40870.491639795444)	Energy: (-40870.49163979546)
Initial perc: (-41027.833837220045)	Final perc:(-41164.79151660442)	Energy: (-41164.79151660442)
Initial perc: (-41027.833837220045)	Final perc:(-41100.79110327455)	Energy: (-41245.26609154223)
Initial perc: (-41027.833837220045)	Final perc:(-41222.49129407629)	Energy: (-41352.4457964965)
Initial perc: (-41027.833837220045)	Final perc:(-41298.004189803025)	Energy: (-41398.64759907913)
Initial perc: (-41027.833837220045)	Final perc:(-41291.96965444586)	Energy: (-41458.57639357576)
Initial perc: (-41027.833837220045)	Final perc

In [13]:
cimsim = CIMSim(J, h.reshape(-1, 1), device='cpu')
cimsim.set_params({'c_th':1., 'zeta':1., 'init_coupling':0.3, 'final_coupling':1., 'N':1000, 'attempt_num':100000, **params[h, J]})
(spins_ising, energy_ising, c_current, c_evol) = cimsim.find_opt()
# solutions[h, J] = spins_ising
print('Im done')

Im done


In [14]:
solution = np.array([int((i+1)/2) for i in spins_ising])
part.permutation_mat = part.list_to_mat(solution)
print("Solution permutation matrix:")
print(part.permutation_mat)
print("\n")

# Check, whether solution matrix is permutation one
if not part.permutation_check(part.permutation_mat):
    print('\033[93m' + "Solution is not permutation matrix. " +
          "Trying to fix it" + '\033[0m')
    new_solution_permutation_mat = part.to_permutation(part.permutation_mat)
    if part.permutation_check(new_solution_permutation_mat):
        print('\033[93m' + "Success!" + '\033[0m')
        part.permutation_mat = new_solution_permutation_mat
    print("Solution matrix after fixing:")
    print(part.permutation_mat)

part.ordered_mat = part.permute(part.permutation_mat, part.mixed_mat)
print("Ordered matrix:")
print(part.ordered_mat)
print("\n")

Solution permutation matrix:
[[0 0 0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0 0 0]]


Solution is not permutation matrix. Trying to fix it
Unable to fix this shit ¯\_(ツ)_/¯
Solution matrix after fixing:
[[0 0 0 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0 0 0]]
Ordered matrix:
[[0.894 0.    0.    0.465 0.    0.693 0.    0.    0.    0.    0.   ]
 [0.    0.898 0.769 0.407 0.232 0.    0.    0.    0.    0.    0.   ]
 [0.    0.769 0.945 0.723 0.557 0.    0.    0.    0.    0.    0.   ]
 [0.465 0.407 0.723 1.758 0.799 0.602 0.    0.    0.    0.    0.   ]
 [0.    

In [45]:
a = 5
print(f"Hello {a}")

Hello 5
